# Phi-4 Mini ONNX समांतर फंक्शन कॉलिंग ट्यूटोरियल

या नोटबुकमध्ये Phi-4 Mini ONNX Runtime GenAI सह समांतर फंक्शन कॉलिंग कसे वापरायचे ते दाखवले आहे. फंक्शन कॉलिंग वापरकर्त्याच्या विनंतीनुसार मॉडेलला बाह्य साधने आणि API बुद्धिमत्तेने वापरण्याची परवानगी देते.

## आढावा

या ट्यूटोरियलमध्ये तुम्ही शिकाल:
- Phi-4 Mini ONNX Runtime GenAI सेट अप कसे करायचे
- फ्लाइट आणि हॉटेल बुकिंगसाठी फंक्शन स्कीमाची व्याख्या कशी करायची
- संरचित आउटपुटसाठी Lark व्याकरणासह मार्गदर्शित जनरेशन कसे वापरायचे
- जटिल प्रवास बुकिंग परिस्थितीसाठी समांतर फंक्शन कॉल कसे अंमलात आणायचे

## पूर्वतयारी

ही नोटबुक चालवण्यापूर्वी, खात्री करा की तुम्ही:
- Phi-4 Mini ONNX मॉडेल डाउनलोड केले आहे
- `onnxruntime-genai` पॅकेज स्थापित केले आहे
- फंक्शन कॉलिंग संकल्पनांची मूलभूत समज आहे


## चरण 1: आवश्यक लायब्ररी आयात करा

सर्वप्रथम, आपल्या फंक्शन कॉलिंग अंमलबजावणीसाठी आवश्यक लायब्ररी आयात करू.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## चरण 2: मॉडेल सेटअप आणि कॉन्फिगरेशन

आता आपण Phi-4 Mini ONNX मॉडेल कॉन्फिगर करू. आपला वास्तविक मॉडेल डिरेक्टरीसह पथ बदलण्याची खात्री करा.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## चरण ३: जनरेशन पॅरामीटर्स कॉन्फिगर करा

मॉडेलच्या आउटपुट वर्तनावर नियंत्रण ठेवण्यासाठी जनरेशन पॅरामीटर्स सेट करा. या सेटिंग्ज फंक्शन कॉलिंगसाठी निश्चित आणि लक्ष केंद्रित प्रतिसाद सुनिश्चित करतात.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## चरण ४: उपलब्ध फंक्शन्स निश्चित करा

येथे आपण आपल्या AI सहाय्यकाला कॉल करता येणाऱ्या फंक्शन्स निश्चित करतो. या उदाहरणात, आमच्याकडे प्रवासाशी संबंधित दोन फंक्शन्स आहेत:
1. **booking_flight_tickets**: विमानतळांदरम्यान फ्लाइट्स बुक करण्यासाठी
2. **booking_hotels**: हॉटेल निवास बुक करण्यासाठी

फंक्शनची व्याख्या OpenAI च्या फंक्शन कॉलिंग स्कीमा फॉरमॅटचे अनुसरण करते.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## चरण ५: व्याकरण निर्मिती सहाय्यक फंक्शन्स

हे सहाय्यक फंक्शन्स आपल्या फंक्शनच्या व्याख्या Lark व्याकरण स्वरूपात रूपांतरित करतात, जे ONNX Runtime GenAI द्वारे मार्गदर्शित निर्मितीसाठी वापरले जाते. यामुळे मॉडेल योग्य JSON स्वरूपात वैध फंक्शन कॉल्स तयार करते.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## चरण 6: व्याकरण निर्मितीची चाचणी करा

आपल्या व्याकरण निर्मिती फंक्शन्सची चाचणी घेऊया, ज्यामुळे आपल्या टूलच्या व्याख्या योग्य स्वरूपात परिवर्तित होतात.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## चरण ७: प्रणाली प्रॉम्प्ट आणि जनरेटर तयार करा

आता आपण प्रणाली प्रॉम्प्ट तयार करू ज्यामध्ये मॉडेलला उपलब्ध साधनांबद्दल माहिती दिली जाते आणि मार्गदर्शित निर्मिती पॅरामीटर्ससह जनरेटर सेट अप करू.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## चरण 8: मार्गदर्शित निर्मितीसाठी जनरेटर प्रारंभ करा

आता आपण आपल्या संरचीत पॅरामीटर्ससह जनरेटर तयार करू आणि मार्गदर्शित निर्मितीसाठी Lark व्याकरण लागू करू.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## चरण 9: समांतर फंक्शन कॉलिंगची चाचणी करा

आता आपण अनेक फंक्शन्स कॉल करण्याची आवश्यकता असलेल्या जटिल प्रवास बुकिंग परिस्थितीसह आपल्या सेटअपची चाचणी करूया.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### फंक्शन कॉल्स तयार करा

मॉडेल आता वापरकर्त्याच्या विनंतीनुसार संरचित फंक्शन कॉल्स तयार करेल. मार्गदर्शित निर्मितीमुळे, आउटपुट वैध JSON स्वरूपात असेल जे थेट कार्यान्वित करता येईल.

**अपेक्षित आउटपुट**: मॉडेलने खालील फंक्शन कॉल्स तयार करावेत:
1. `booking_flight_tickets` बीजिंग (PEK) ते पॅरिस (CDG) तपशीलांसह
2. `booking_hotels` पॅरिस निवासस्थानाच्या तपशीलांसह

खालील सेल चालवा आणि थेट निर्मिती पहा:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## निष्कर्ष

🎉 **अभिनंदन!** तुम्ही ONNX Runtime GenAI वापरून Phi-4 Mini सह समांतर फंक्शन कॉलिंग यशस्वीपणे अंमलात आणले आहे.

### तुम्ही काय शिकलात:

1. **मॉडेल सेटअप**: Phi-4 Mini ONNX Runtime GenAI सह कसे कॉन्फिगर करायचे
2. **फंक्शन परिभाषा**: AI फंक्शन कॉलिंगसाठी फंक्शन स्कीमाची परिभाषा कशी करायची
3. **मार्गदर्शित जनरेशन**: संरचित आउटपुट जनरेशनसाठी Lark व्याकरणाचा वापर कसा करायचा
4. **समांतर फंक्शन कॉल्स**: अनेक फंक्शन कॉल्स आवश्यक असलेल्या जटिल विनंत्यांचे व्यवस्थापन कसे करायचे

### मुख्य फायदे:

- ✅ **संरचित आउटपुट**: मार्गदर्शित जनरेशनमुळे वैध JSON फंक्शन कॉल्स सुनिश्चित होतात
- ✅ **समांतर प्रक्रिया**: एका विनंतीमध्ये अनेक फंक्शन कॉल्स हाताळा
- ✅ **उच्च कार्यक्षमता**: ONNX Runtime अनुकूलित अनुमान प्रदान करते
- ✅ **लवचिक स्कीमा**: फंक्शन परिभाषा सहजपणे जोडणे किंवा बदलणे शक्य

### संसाधने:

- [Phi-4 Mini दस्तऐवज](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [ONNX Runtime GenAI दस्तऐवज](https://onnxruntime.ai/docs/genai/)
- [फंक्शन कॉलिंग सर्वोत्तम पद्धती](https://platform.openai.com/docs/guides/function-calling)



---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) चा वापर करून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी कृपया लक्षात ठेवा की स्वयंचलित भाषांतरे त्रुटी किंवा अचूकतेच्या अभावाने युक्त असू शकतात. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर करून उद्भवलेल्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
